In [24]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [25]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
csfont = {'fontname':'Times New Roman'}

#Lists
#RLength
ThetaList=['0.0','22.5','45.0','67.5','90.0','112.5','135.0','157.5','180.0',
           '202.5','225.0','247.5','270.0','292.5','315.0','337.5']
HxList=['0.5','1.5','2.5','3.5','4.5','5.5','6.5','7.5','8.5','9.5','10.5','11.5','12.5']
HyList=['-13','-11','-9','-7','-5','-3','-1','1','3','5','7','9']

allData = []

In [26]:
# constructs a filepath for the pos data of Re = $Re
def pname(cwd):
    return cwd+"/NOTA_List.txt"

def csvname(cwd,rd):
    if(rd == '1'):
        return cwd+'/LTT_allData_Re2.csv'
    elif(rd == '2'):
        return cwd+'/LTT_allData_Re2_Rd2.csv'

def GetNOTAData(cwd):
    data = pd.read_csv(pname(cwd),delimiter=' ')
    data = data.sort_values(by=['Theta','parHx','parHy'])
    data = data.reset_index(drop=True)
    return data

def GetRd1CSV(cwd):
    data = pd.read_csv(csvname(cwd,'1'),delimiter=' ')
    data = data.sort_values(by=['parThetaBW','parHx','parHy','time'])
    data = data.reset_index(drop=True)
    return data

def GetRd2CSV(cwd):
    data = pd.read_csv(csvname(cwd,'2'),delimiter=' ')
    data = data.sort_values(by=['parThetaBW','parHx','parHy','time'])
    data = data.reset_index(drop=True)
    return data

In [27]:
#Get Rd1 csv data
dataRd1 = GetRd1CSV(cwd_PYTHON)
print('Rd1 CSV gotten')
#Get Rd2 csv data
dataRd2 = GetRd2CSV(cwd_PYTHON)
print('Rd2 CSV gotten')
allData = pd.concat([dataRd1,dataRd2],ignore_index=True)
allData = allData.sort_values(by=['parThetaBW','parHx','parHy','time'])
allData = allData.reset_index(drop=True)
print('allData created and sorted')

Rd1 CSV gotten
Rd2 CSV gotten
allData created and sorted


In [28]:
import matplotlib
def WhichEndState(data,scheme):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0
    #V-Shape = 1
    #In-line = 2
    #Headbutt = 3
    #L-shape = 4
    #NOTA = 5
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 54 or 306; H_bw ~ 3.8R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #In-line: Theta = 0, 360; H_bw = 4.6-4.7R
    #L-Shape: Theta = 95-100, 260-265; H_bw = 4.5-4.6R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 5
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['ThetaBW']/np.pi
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    color_value = matplotlib.colors.to_rgb('black')
    for idx in range(len(data['time'])):
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
            '''
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 255
                data.loc[idx,'color_g'] = 217
                data.loc[idx,'color_b'] = 47
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 230
                data.loc[idx,'color_g'] = 171
                data.loc[idx,'color_b'] = 2
            '''
        #V-Shape
        elif((data.loc[idx,'H'] < 4.9 and data.loc[idx,'H'] >= 4.7) and 
             ((data.loc[idx,'Theta_deg'] < 60.0 and data.loc[idx,'Theta_deg'] >= 50.0) or
              (data.loc[idx,'Theta_deg'] < 310.0 and data.loc[idx,'Theta_deg'] >= 300.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            #color_value = matplotlib.colors.to_rgb('blue')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 141
                data.loc[idx,'color_g'] = 160
                data.loc[idx,'color_b'] = 203
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 117
                data.loc[idx,'color_g'] = 112
                data.loc[idx,'color_b'] = 179
            
        #In-Line
        elif((data.loc[idx,'H'] < 4.7 and data.loc[idx,'H'] >= 4.5) and 
             ((data.loc[idx,'Theta_deg'] < 5.0 and data.loc[idx,'Theta_deg'] >= 0.0) or
              (data.loc[idx,'Theta_deg'] < 360.0 and data.loc[idx,'Theta_deg'] >= 355.0))):
            data.loc[idx,'State'] = 2
            data.loc[idx,'color'] = 'green'
            #color_value = matplotlib.colors.to_rgb('green')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 102
                data.loc[idx,'color_g'] = 194
                data.loc[idx,'color_b'] = 165
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 27
                data.loc[idx,'color_g'] = 158
                data.loc[idx,'color_b'] = 119
        #Headbutt
        elif((data.loc[idx,'H'] < 3.5 and data.loc[idx,'H'] >= 3.3) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            #color_value = matplotlib.colors.to_rgb('orange')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 252
                data.loc[idx,'color_g'] = 141
                data.loc[idx,'color_b'] = 98
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 217
                data.loc[idx,'color_g'] = 95
                data.loc[idx,'color_b'] = 2
        #L-Shape (Metastable)
        elif((data.loc[idx,'H'] < 4.6 and data.loc[idx,'H'] >= 4.4) and 
             ((data.loc[idx,'Theta_deg'] < 100.0 and data.loc[idx,'Theta_deg'] >= 95.0) or
              (data.loc[idx,'Theta_deg'] < 265.0 and data.loc[idx,'Theta_deg'] >= 260.0))):
            data.loc[idx,'State'] = 4
            data.loc[idx,'color'] = 'green'
            #color_value = matplotlib.colors.to_rgb('green')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 102
                data.loc[idx,'color_g'] = 194
                data.loc[idx,'color_b'] = 165
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 27
                data.loc[idx,'color_g'] = 158
                data.loc[idx,'color_b'] = 119
        else:
            color_value= matplotlib.colors.to_rgb('black')
        #data.loc[idx,'color_r'] = color_value[0]
        #data.loc[idx,'color_g'] = color_value[1]
        #data.loc[idx,'color_b'] = color_value[2]
    
    return data

In [30]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

endDict = {'parThetaBW':[],'parHx':[],'parHy':[],'ThetaBW':[],'Hx':[],'Hy':[],'time':[],'H':[]}
endData = pd.DataFrame(data=endDict)


#print(len(allData))
for Theta in ThetaList:
    print('Theta = ',Theta)
    thetaData = allData[allData['parThetaBW'] <= float(Theta)+0.01].copy()
    thetaData = thetaData[thetaData['parThetaBW'] >= float(Theta)-0.01].copy()
    #print(len(thetaData['time']))
    for Hx in HxList:
        #print(Hx)
        hxData = thetaData[thetaData['parHx'] <= float(Hx)/2.0 + 0.01].copy()
        hxData = hxData[hxData['parHx'] >= float(Hx)/2.0 - 0.01].copy()
        #print(len(hxData['time']))
        dirNames = [ name for name in os.listdir(cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/') 
                    if os.path.isdir(os.path.join(cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/', name)) ]
        #print(hxData['parHy'].values)
        #print('dirHys = ',dirNames)
        for dirHy in dirNames:
            #print(dirHy)
            HyString, HyValue = tuple(re.split('Hy',dirHy)[:2])
            HyValue2 = round(float(HyValue)/2.0,1)
            #print(HyValue2)
            hyData = hxData[hxData['parHy'] <= HyValue2+0.01].copy()
            hyData = hyData[hyData['parHy'] >= HyValue2-0.01].copy()
            hyData = hyData.sort_values(by=['time'])
            hyData = hyData.reset_index(drop=True)
            #print(len(hyData['time']))
            #Get last timestep
            #simData = endData.loc[-1,'time'].copy()
            simData = hyData.tail(1)
            #print(hyData.tail())
            simData = simData.reset_index(drop=True)
            simDict = {'parThetaBW':simData['parThetaBW'].values.tolist(),'parHx':simData['parHx'].values.tolist(),'parHy':simData['parHy'].values.tolist(),
                       'ThetaBW':simData['ThetaBW'].values.tolist(),'Hx':simData['Hx'].values.tolist(),'Hy':simData['Hy'].values.tolist(),
                       'time':simData['time'].values.tolist(),'H':simData['H'].values.tolist()}
            data = pd.DataFrame(data=simDict)
            #print(data)
            #print('%s: %s: %.2f: %.1f'%(Theta,Hx,HyValue2,data.loc[0,'time']))
            endData = pd.concat([endData,data],ignore_index=True)

endData = endData.sort_values(by=['parThetaBW','parHx','parHy'])
endData = endData.reset_index(drop=True)

endData['xval'] = endData['parHx']*np.cos(endData['parThetaBW']*np.pi/180.0)
endData['yval'] = endData['parHx']*np.sin(endData['parThetaBW']*np.pi/180.0)
endData['zval'] = endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
scheme='dark'
endData = WhichEndState(endData,scheme)

#Drop NOTA sims
endData = endData[endData['color'] != 'black'].copy()
endData = endData.reset_index(drop=True)

endData['ColorString'] = 0.0
for idx in range(len(endData['color_r'])):
    endData.loc[idx,'ColorString'] = 'rgb({0},{1},{2})'.format(endData.loc[idx,'color_r'],endData.loc[idx,'color_g'],endData.loc[idx,'color_b'])


#Diverge = 0
#V-Shape = 1
#In-line = 2
#Headbutt = 3
#L-shape = 4
#NOTA = 5
    
#Create traces for each end state
traces = [0 for a in range(8)]
opaque = 0.4

#Divergence
divergeData = endData[endData['State'] == 0].copy()
divergeData = divergeData.reset_index(drop=True)
divColorString = 'rgb({0},{1},{2})'.format(divergeData.loc[0,'color_r'],divergeData.loc[0,'color_g'],divergeData.loc[0,'color_b'])
traces[0] = go.Mesh3d(x=divergeData['xval'], 
                     y=divergeData['yval'], 
                     z=divergeData['zval'], 
                     #color=divergeData.loc[0,'color'],
                     color=divergeData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#V-Shape
VshapeData = endData[endData['State'] == 1].copy()
VshapeData = VshapeData.reset_index(drop=True)
VColorString = 'rgb({0},{1},{2})'.format(VshapeData.loc[0,'color_r'],VshapeData.loc[0,'color_g'],VshapeData.loc[0,'color_b'])
traces[1] = go.Mesh3d(x=VshapeData['xval'], 
                     y=VshapeData['yval'], 
                     z=VshapeData['zval'], 
                     #color=VshapeData.loc[0,'color'], 
                     color=VshapeData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5.25
                     )

#In-Line
inLineData = endData[endData['State'] == 2].copy()
inLineData = inLineData.reset_index(drop=True)
inLineColorString = 'rgb({0},{1},{2})'.format(inLineData.loc[0,'color_r'],inLineData.loc[0,'color_g'],inLineData.loc[0,'color_b'])

inLine1Data = inLineData[inLineData['parThetaBW'] <= 180.0].copy()
inLine1Data = inLine1Data.reset_index(drop=True)
inLine2Data = inLineData[inLineData['parThetaBW'] > 180.0].copy()
inLine2Data = inLine2Data.reset_index(drop=True)
inLine3Data = inLineData[inLineData['zval'] >= 3.5].copy()
inLine3Data = inLine3Data.reset_index(drop=True)
inLine4Data = inLineData[inLineData['zval'] <= -3.5].copy()
inLine4Data = inLine4Data.reset_index(drop=True)
inLine5Data = inLineData[inLineData['zval'] >= -3.5].copy()
inLine5Data = inLine5Data[inLine5Data['zval'] <= 3.5].copy()
inLine5Data = inLine5Data.reset_index(drop=True)
inLine6Data = inLine5Data[inLine5Data['parThetaBW'] <= 180.0].copy()
inLine6Data = inLine6Data.reset_index(drop=True)
inLine7Data = inLine5Data[inLine5Data['parThetaBW'] > 180.0].copy()
inLine7Data = inLine7Data.reset_index(drop=True)

traces[2] = go.Mesh3d(x=inLine3Data['xval'], 
                     y=inLine3Data['yval'], 
                     z=inLine3Data['zval'], 
                     #color=inLine3Data.loc[0,'color'],
                     color=inLine3Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=0.2,
                     alphahull=5
                     )

traces[3] = go.Mesh3d(x=inLine4Data['xval'], 
                     y=inLine4Data['yval'], 
                     z=inLine4Data['zval'], 
                     #color=inLine4Data.loc[0,'color'],
                     color=inLine4Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=0.2,
                     alphahull=9
                     )

traces[4] = go.Mesh3d(x=inLine6Data['xval'], 
                     y=inLine6Data['yval'], 
                     z=inLine6Data['zval'], 
                     #color=inLine6Data.loc[0,'color'], 
                     color=inLine6Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=0.2,
                     alphahull=5
                     )

traces[5] = go.Mesh3d(x=inLine7Data['xval'], 
                     y=inLine7Data['yval'], 
                     z=inLine7Data['zval'], 
                     #color=inLine7Data.loc[0,'color'], 
                     color=inLine7Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=0.2,
                     alphahull=6
                     )

#Headbutt
headbuttData = endData[endData['State'] == 3].copy()
headbuttData = headbuttData.reset_index(drop=True)
HBColorString = 'rgb({0},{1},{2})'.format(headbuttData.loc[0,'color_r'],headbuttData.loc[0,'color_g'],headbuttData.loc[0,'color_b'])
traces[6] = go.Mesh3d(x=headbuttData['xval'], 
                     y=headbuttData['yval'], 
                     z=headbuttData['zval'], 
                     #color=headbuttData.loc[0,'color'], 
                     color=headbuttData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=0
                     )

traces[7] = go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=12,
        #color=endData['color'],                # set color to an array/list of desired values
        color=endData['ColorString'],
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=10)
    )
)

layout = go.Layout(
    #title='Re10: 3D Phase Diagram',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    showlegend=False,
)

fig = go.Figure(data = traces, layout=layout)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene=dict(
                    #xaxis_title=r'Hx*cos(theta) (R)',
                    #yaxis_title=r'Hx*sin(theta) (R)',
                    #zaxis_title=r'Hy (R)'),
                    #xaxis_title='H<sub>x<\sub>cos(&theta) (R)',
                    #yaxis_title='H<sub>x<\sub>sin(&theta) (R)',
                    #zaxis_title='H<sub>y<\sub> (R)',
                    xaxis_title=dict(text="H<sub>x</sub> cos("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    yaxis_title=dict(text="H<sub>x</sub> sin("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    zaxis_title=dict(text="H<sub>y</sub> (R)",font=dict(size=18,family='Times New Roman')),
                    ),
                    width=2000,height=2000,
                    margin=dict(r=100, b=100, l=100, t=100),
                    font=dict(
                            family="Times New Roman",
                            size=12,
                            color="black"
                            )
            )
#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_zaxes(automargin=True)

camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()


if not os.path.exists(cwd_PYTHON+'/../Figures/3D_Diagram'):
    os.mkdir(cwd_PYTHON+'/../Figures/3D_Diagram')
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/allRe2_topview.png')
print('Done Making Top View')
camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=-2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/allRe2_botview.png')
print('Done Making Bot View')

camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=2.0, z=0.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()

fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/allRe2_yposview.png')
print('Done Making Ypos View')

camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=-2.0, z=0.0),
    #center=dict(x=0, y=0, z=1.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/allRe2_ynegview.png')
print('Done Making Yneg View')



Theta =  0.0
Theta =  22.5
Theta =  45.0
Theta =  67.5
Theta =  90.0
Theta =  112.5
Theta =  135.0
Theta =  157.5
Theta =  180.0
Theta =  202.5
Theta =  225.0
Theta =  247.5
Theta =  270.0
Theta =  292.5
Theta =  315.0
Theta =  337.5


ValueError: 
For some reason plotly.py was unable to communicate with the
local orca server process, even though the server process seems to be running.

Please review the process and connection information below:

orca status
-----------
    state: running
    executable: /Users/thomas/anaconda3/envs/tf-research/bin/orca
    version: 1.2.1
    port: 56109
    pid: 4567
    command: ['/Users/thomas/anaconda3/envs/tf-research/bin/orca', 'serve', '-p', '56109', '--plotly', '/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/plotly/package_data/plotly.min.js', '--graph-only', '--mathjax', 'https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js']




In [50]:
%matplotlib inline

def Construct_Image_Data(data):
    #From our color data and initial configurations, construct 2D mesh which has positions and colors for each position
    #If parHx and parHy do not exist, make color white
    xval = np.linspace(-0.5,12.5,14)
    yval = np.linspace(-13.0,9.0,12)
    #print(xval)
    #print(yval)
    
    image = np.ones((12,14,3)) #idx, jdx, r, g, b
    
    for idx in range(len(xval)):
        HxData = data[data['parHx'] == xval[idx]].copy()
        for jdx in range(len(yval)):
            df = HxData[HxData['parHy'] == yval[jdx]].copy()
            #print(len(df['time']))
            if(len(df['time']) == 1):
                image[jdx,idx,:] = [df['color_r'].values,df['color_g'].values,df['color_b'].values]
                
            else:
                image[jdx,idx,:] = [matplotlib.colors.to_rgb('white')[0],matplotlib.colors.to_rgb('white')[1],matplotlib.colors.to_rgb('white')[2]]
    
    return image

endDict = {'parThetaBW':[],'parHx':[],'parHy':[],'ThetaBW':[],'Hx':[],'Hy':[],'Nosc':[],'State':[],'vx':[],'vy':[]}
endData = pd.DataFrame(data=endDict)

#print(len(allData))
for Theta in ThetaList:
    print('Theta = ',Theta)
    thetaData = allData[allData['parThetaBW'] <= float(Theta)+0.01].copy()
    thetaData = thetaData[thetaData['parThetaBW'] >= float(Theta)-0.01].copy()
    #print(len(thetaData['time']))
    for Hx in HxList:
        #print(Hx)
        hxData = thetaData[thetaData['parHx'] <= float(Hx)/2.0 + 0.01].copy()
        hxData = hxData[hxData['parHx'] >= float(Hx)/2.0 - 0.01].copy()
        #print(len(hxData['time']))
        dirNames = [ name for name in os.listdir(cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/') 
                    if os.path.isdir(os.path.join(cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/', name)) ]
        #print(hxData['parHy'].values)
        #print('dirHys = ',dirNames)
        for dirHy in dirNames:
            #print(dirHy)
            HyString, HyValue = tuple(re.split('Hy',dirHy)[:2])
            HyValue2 = round(float(HyValue)/2.0,1)
            #print(HyValue2)
            hyData = hxData[hxData['parHy'] <= HyValue2+0.01].copy()
            hyData = hyData[hyData['parHy'] >= HyValue2-0.01].copy()
            hyData = hyData.sort_values(by=['time'])
            hyData = hyData.reset_index(drop=True)
            
            simData = WhichEndState(hyData,scheme)
            
            lastData = hyData.tail(1)
            lastIndexList = lastData.index.values
            lastIndex = lastIndexList[0]
            if(lastIndex != 200):
                print('LastIndex: ',lastIndex)
            #print(lastIndex)
            #Find when Simulation is in end state
            scheme='dark'
            count = 5
            boolEnd = 0
            for count in range(5,lastIndex-5):
            #while(count < lastIndex-5):
                State = simData.loc[count,'State']
                if(State != 5):
                    b4State = simData.loc[count-5,'State']
                    a4State = simData.loc[lastIndex-5,'State']
                    if(State == b4State and State == a4State):
                        #Simulation is in end state
                        print('End State! Theta: {0}, Hx: {1}, Hy:{2}, Nosc: {3}'.format(Theta,float(Hx)/2.0,HyValue2,count))
                        #Create a database with sim information
                        simDict = {'parThetaBW':[simData.loc[count,'parThetaBW']],'parHx':[simData.loc[count,'parHx']],
                                   'parHy':[simData.loc[count,'parHy']],'ThetaBW':[simData.loc[count,'ThetaBW']],
                                   'Hx':[simData.loc[count,'Hx']],'Hy':[simData.loc[count,'Hy']],'Nosc':[count],'State':[simData.loc[count,'State']]}
                        simData = pd.DataFrame(data=simDict)
                        simData['vx'] = (simData['Hx']-simData['parHx'])/simData['Nosc']
                        simData['vy'] = (simData['Hy']-simData['parHy'])/simData['Nosc']
                        endData = pd.concat([endData,simData],ignore_index=True)
                        boolEnd = 1
                        break
            if(boolEnd == 0):
                print('No End State! Theta: {0}, Hx: {1}, Hy:{2}, Nosc: {3}'.format(Theta,float(Hx)/2.0,HyValue2,count))
endData = endData.sort_values(by=['parThetaBW','parHx','parHy'])
endData = endData.reset_index(drop=True)
print('Done!')
                    

Theta =  0.0
End State! Theta: 0.0, Hx: 0.25, Hy:-5.5, Nosc: 11
End State! Theta: 0.0, Hx: 0.25, Hy:-6.5, Nosc: 17
End State! Theta: 0.0, Hx: 0.25, Hy:-4.5, Nosc: 5
End State! Theta: 0.0, Hx: 0.25, Hy:4.5, Nosc: 5
End State! Theta: 0.0, Hx: 0.75, Hy:-5.5, Nosc: 12
End State! Theta: 0.0, Hx: 0.75, Hy:-6.5, Nosc: 18
End State! Theta: 0.0, Hx: 0.75, Hy:-4.5, Nosc: 5
End State! Theta: 0.0, Hx: 0.75, Hy:4.5, Nosc: 5
End State! Theta: 0.0, Hx: 1.25, Hy:-5.5, Nosc: 13
End State! Theta: 0.0, Hx: 1.25, Hy:-6.5, Nosc: 19
End State! Theta: 0.0, Hx: 1.25, Hy:-4.5, Nosc: 7
End State! Theta: 0.0, Hx: 1.25, Hy:4.5, Nosc: 7
End State! Theta: 0.0, Hx: 1.75, Hy:-5.5, Nosc: 14
End State! Theta: 0.0, Hx: 1.75, Hy:-6.5, Nosc: 21
End State! Theta: 0.0, Hx: 1.75, Hy:-4.5, Nosc: 14
End State! Theta: 0.0, Hx: 1.75, Hy:4.5, Nosc: 13
End State! Theta: 0.0, Hx: 2.25, Hy:-0.5, Nosc: 91
End State! Theta: 0.0, Hx: 2.25, Hy:-5.5, Nosc: 17
End State! Theta: 0.0, Hx: 2.25, Hy:-6.5, Nosc: 23
End State! Theta: 0.0, Hx: 2

End State! Theta: 22.5, Hx: 3.25, Hy:-0.5, Nosc: 78
End State! Theta: 22.5, Hx: 3.25, Hy:-5.5, Nosc: 27
End State! Theta: 22.5, Hx: 3.25, Hy:-6.5, Nosc: 34
End State! Theta: 22.5, Hx: 3.25, Hy:-1.5, Nosc: 73
End State! Theta: 22.5, Hx: 3.25, Hy:-2.5, Nosc: 67
End State! Theta: 22.5, Hx: 3.25, Hy:-3.5, Nosc: 23
End State! Theta: 22.5, Hx: 3.25, Hy:-4.5, Nosc: 23
End State! Theta: 22.5, Hx: 3.25, Hy:0.5, Nosc: 78
End State! Theta: 22.5, Hx: 3.25, Hy:1.5, Nosc: 74
End State! Theta: 22.5, Hx: 3.25, Hy:2.5, Nosc: 67
End State! Theta: 22.5, Hx: 3.25, Hy:3.5, Nosc: 60
End State! Theta: 22.5, Hx: 3.25, Hy:4.5, Nosc: 56
End State! Theta: 22.5, Hx: 3.75, Hy:-0.5, Nosc: 72
End State! Theta: 22.5, Hx: 3.75, Hy:-5.5, Nosc: 31
End State! Theta: 22.5, Hx: 3.75, Hy:-6.5, Nosc: 38
End State! Theta: 22.5, Hx: 3.75, Hy:-1.5, Nosc: 69
End State! Theta: 22.5, Hx: 3.75, Hy:-2.5, Nosc: 63
End State! Theta: 22.5, Hx: 3.75, Hy:-3.5, Nosc: 98
End State! Theta: 22.5, Hx: 3.75, Hy:-4.5, Nosc: 28
End State! Theta:

End State! Theta: 45.0, Hx: 4.75, Hy:-2.5, Nosc: 57
End State! Theta: 45.0, Hx: 4.75, Hy:-3.5, Nosc: 54
End State! Theta: 45.0, Hx: 4.75, Hy:-4.5, Nosc: 43
End State! Theta: 45.0, Hx: 4.75, Hy:0.5, Nosc: 63
End State! Theta: 45.0, Hx: 4.75, Hy:1.5, Nosc: 60
End State! Theta: 45.0, Hx: 4.75, Hy:2.5, Nosc: 56
End State! Theta: 45.0, Hx: 4.75, Hy:3.5, Nosc: 48
End State! Theta: 45.0, Hx: 4.75, Hy:4.5, Nosc: 43
End State! Theta: 45.0, Hx: 5.25, Hy:-0.5, Nosc: 56
End State! Theta: 45.0, Hx: 5.25, Hy:-5.5, Nosc: 55
End State! Theta: 45.0, Hx: 5.25, Hy:-6.5, Nosc: 61
End State! Theta: 45.0, Hx: 5.25, Hy:-1.5, Nosc: 54
End State! Theta: 45.0, Hx: 5.25, Hy:-2.5, Nosc: 54
End State! Theta: 45.0, Hx: 5.25, Hy:-3.5, Nosc: 73
End State! Theta: 45.0, Hx: 5.25, Hy:-4.5, Nosc: 54
End State! Theta: 45.0, Hx: 5.25, Hy:0.5, Nosc: 56
End State! Theta: 45.0, Hx: 5.25, Hy:1.5, Nosc: 55
End State! Theta: 45.0, Hx: 5.25, Hy:2.5, Nosc: 51
End State! Theta: 45.0, Hx: 5.25, Hy:3.5, Nosc: 45
End State! Theta: 45.

End State! Theta: 67.5, Hx: 6.25, Hy:-4.5, Nosc: 87
End State! Theta: 67.5, Hx: 6.25, Hy:0.5, Nosc: 45
End State! Theta: 67.5, Hx: 6.25, Hy:1.5, Nosc: 44
End State! Theta: 67.5, Hx: 6.25, Hy:2.5, Nosc: 41
End State! Theta: 67.5, Hx: 6.25, Hy:3.5, Nosc: 37
End State! Theta: 67.5, Hx: 6.25, Hy:4.5, Nosc: 30
Theta =  90.0
End State! Theta: 90.0, Hx: 0.25, Hy:-5.5, Nosc: 93
End State! Theta: 90.0, Hx: 0.25, Hy:-6.5, Nosc: 46
LastIndex:  401
End State! Theta: 90.0, Hx: 0.25, Hy:-4.5, Nosc: 36
End State! Theta: 90.0, Hx: 0.25, Hy:3.5, Nosc: 141
End State! Theta: 90.0, Hx: 0.25, Hy:4.5, Nosc: 75
End State! Theta: 90.0, Hx: 0.75, Hy:-5.5, Nosc: 78
End State! Theta: 90.0, Hx: 0.75, Hy:-6.5, Nosc: 45
LastIndex:  401
End State! Theta: 90.0, Hx: 0.75, Hy:-4.5, Nosc: 12
End State! Theta: 90.0, Hx: 0.75, Hy:3.5, Nosc: 111
End State! Theta: 90.0, Hx: 0.75, Hy:4.5, Nosc: 77
End State! Theta: 90.0, Hx: 1.25, Hy:-5.5, Nosc: 56
End State! Theta: 90.0, Hx: 1.25, Hy:-6.5, Nosc: 45
End State! Theta: 90.0, H

End State! Theta: 112.5, Hx: 2.75, Hy:-5.5, Nosc: 62
End State! Theta: 112.5, Hx: 2.75, Hy:-6.5, Nosc: 60
End State! Theta: 112.5, Hx: 2.75, Hy:-3.5, Nosc: 57
End State! Theta: 112.5, Hx: 2.75, Hy:-4.5, Nosc: 28
End State! Theta: 112.5, Hx: 2.75, Hy:1.5, Nosc: 23
End State! Theta: 112.5, Hx: 2.75, Hy:2.5, Nosc: 49
End State! Theta: 112.5, Hx: 2.75, Hy:3.5, Nosc: 133
End State! Theta: 112.5, Hx: 2.75, Hy:4.5, Nosc: 70
End State! Theta: 112.5, Hx: 3.25, Hy:-0.5, Nosc: 101
End State! Theta: 112.5, Hx: 3.25, Hy:-5.5, Nosc: 58
End State! Theta: 112.5, Hx: 3.25, Hy:-6.5, Nosc: 65
End State! Theta: 112.5, Hx: 3.25, Hy:-1.5, Nosc: 126
End State! Theta: 112.5, Hx: 3.25, Hy:-2.5, Nosc: 60
End State! Theta: 112.5, Hx: 3.25, Hy:-3.5, Nosc: 34
End State! Theta: 112.5, Hx: 3.25, Hy:-4.5, Nosc: 92
End State! Theta: 112.5, Hx: 3.25, Hy:0.5, Nosc: 52
End State! Theta: 112.5, Hx: 3.25, Hy:1.5, Nosc: 53
End State! Theta: 112.5, Hx: 3.25, Hy:2.5, Nosc: 104
End State! Theta: 112.5, Hx: 3.25, Hy:3.5, Nosc: 

...

End State! Theta: 225.0, Hx: 3.75, Hy:2.5, Nosc: 83
End State! Theta: 225.0, Hx: 3.75, Hy:3.5, Nosc: 75
End State! Theta: 225.0, Hx: 3.75, Hy:4.5, Nosc: 45
End State! Theta: 225.0, Hx: 4.25, Hy:-0.5, Nosc: 145
End State! Theta: 225.0, Hx: 4.25, Hy:-5.5, Nosc: 50
End State! Theta: 225.0, Hx: 4.25, Hy:-6.5, Nosc: 54
End State! Theta: 225.0, Hx: 4.25, Hy:-1.5, Nosc: 128
End State! Theta: 225.0, Hx: 4.25, Hy:-2.5, Nosc: 133
End State! Theta: 225.0, Hx: 4.25, Hy:-3.5, Nosc: 51
End State! Theta: 225.0, Hx: 4.25, Hy:-4.5, Nosc: 47
End State! Theta: 225.0, Hx: 4.25, Hy:0.5, Nosc: 191
End State! Theta: 225.0, Hx: 4.25, Hy:1.5, Nosc: 108
End State! Theta: 225.0, Hx: 4.25, Hy:2.5, Nosc: 90
End State! Theta: 225.0, Hx: 4.25, Hy:3.5, Nosc: 78
End State! Theta: 225.0, Hx: 4.25, Hy:4.5, Nosc: 44
End State! Theta: 225.0, Hx: 4.75, Hy:-0.5, Nosc: 147
End State! Theta: 225.0, Hx: 4.75, Hy:-5.5, Nosc: 53
End State! Theta: 225.0, Hx: 4.75, Hy:-6.5, Nosc: 55
End State! Theta: 225.0, Hx: 4.75, Hy:-1.5, Nosc

End State! Theta: 247.5, Hx: 5.75, Hy:-5.5, Nosc: 53
End State! Theta: 247.5, Hx: 5.75, Hy:-6.5, Nosc: 57
End State! Theta: 247.5, Hx: 5.75, Hy:-1.5, Nosc: 92
End State! Theta: 247.5, Hx: 5.75, Hy:-2.5, Nosc: 97
End State! Theta: 247.5, Hx: 5.75, Hy:-3.5, Nosc: 47
End State! Theta: 247.5, Hx: 5.75, Hy:-4.5, Nosc: 50
End State! Theta: 247.5, Hx: 5.75, Hy:0.5, Nosc: 94
End State! Theta: 247.5, Hx: 5.75, Hy:1.5, Nosc: 66
End State! Theta: 247.5, Hx: 5.75, Hy:2.5, Nosc: 58
End State! Theta: 247.5, Hx: 5.75, Hy:3.5, Nosc: 76
End State! Theta: 247.5, Hx: 5.75, Hy:4.5, Nosc: 127
End State! Theta: 247.5, Hx: 6.25, Hy:-0.5, Nosc: 61
End State! Theta: 247.5, Hx: 6.25, Hy:-5.5, Nosc: 56
End State! Theta: 247.5, Hx: 6.25, Hy:-6.5, Nosc: 60
End State! Theta: 247.5, Hx: 6.25, Hy:-1.5, Nosc: 63
End State! Theta: 247.5, Hx: 6.25, Hy:-2.5, Nosc: 69
End State! Theta: 247.5, Hx: 6.25, Hy:-3.5, Nosc: 51
End State! Theta: 247.5, Hx: 6.25, Hy:-4.5, Nosc: 52
End State! Theta: 247.5, Hx: 6.25, Hy:0.5, Nosc: 5

End State! Theta: 292.5, Hx: 1.75, Hy:4.5, Nosc: 32
End State! Theta: 292.5, Hx: 2.25, Hy:-5.5, Nosc: 41
End State! Theta: 292.5, Hx: 2.25, Hy:-6.5, Nosc: 66
End State! Theta: 292.5, Hx: 2.25, Hy:-3.5, Nosc: 49
End State! Theta: 292.5, Hx: 2.25, Hy:-4.5, Nosc: 10
End State! Theta: 292.5, Hx: 2.25, Hy:3.5, Nosc: 21
End State! Theta: 292.5, Hx: 2.25, Hy:4.5, Nosc: 31
End State! Theta: 292.5, Hx: 2.75, Hy:-5.5, Nosc: 42
End State! Theta: 292.5, Hx: 2.75, Hy:-6.5, Nosc: 58
End State! Theta: 292.5, Hx: 2.75, Hy:-3.5, Nosc: 28
End State! Theta: 292.5, Hx: 2.75, Hy:-4.5, Nosc: 16
End State! Theta: 292.5, Hx: 2.75, Hy:3.5, Nosc: 22
End State! Theta: 292.5, Hx: 2.75, Hy:4.5, Nosc: 30
End State! Theta: 292.5, Hx: 3.25, Hy:-5.5, Nosc: 44
End State! Theta: 292.5, Hx: 3.25, Hy:-6.5, Nosc: 57
End State! Theta: 292.5, Hx: 3.25, Hy:-3.5, Nosc: 17
End State! Theta: 292.5, Hx: 3.25, Hy:-4.5, Nosc: 22
End State! Theta: 292.5, Hx: 3.25, Hy:3.5, Nosc: 25
End State! Theta: 292.5, Hx: 3.25, Hy:4.5, Nosc: 30


End State! Theta: 315.0, Hx: 4.75, Hy:-2.5, Nosc: 32
End State! Theta: 315.0, Hx: 4.75, Hy:-3.5, Nosc: 56
End State! Theta: 315.0, Hx: 4.75, Hy:-4.5, Nosc: 69
End State! Theta: 315.0, Hx: 4.75, Hy:0.5, Nosc: 40
End State! Theta: 315.0, Hx: 4.75, Hy:1.5, Nosc: 49
End State! Theta: 315.0, Hx: 4.75, Hy:2.5, Nosc: 26
End State! Theta: 315.0, Hx: 4.75, Hy:3.5, Nosc: 26
End State! Theta: 315.0, Hx: 4.75, Hy:4.5, Nosc: 28
End State! Theta: 315.0, Hx: 5.25, Hy:-0.5, Nosc: 36
End State! Theta: 315.0, Hx: 5.25, Hy:-5.5, Nosc: 89
End State! Theta: 315.0, Hx: 5.25, Hy:-6.5, Nosc: 101
End State! Theta: 315.0, Hx: 5.25, Hy:-1.5, Nosc: 36
End State! Theta: 315.0, Hx: 5.25, Hy:-2.5, Nosc: 52
End State! Theta: 315.0, Hx: 5.25, Hy:-3.5, Nosc: 61
End State! Theta: 315.0, Hx: 5.25, Hy:-4.5, Nosc: 75
End State! Theta: 315.0, Hx: 5.25, Hy:0.5, Nosc: 41
End State! Theta: 315.0, Hx: 5.25, Hy:1.5, Nosc: 51
End State! Theta: 315.0, Hx: 5.25, Hy:2.5, Nosc: 29
End State! Theta: 315.0, Hx: 5.25, Hy:3.5, Nosc: 29
E

End State! Theta: 337.5, Hx: 6.25, Hy:-3.5, Nosc: 163
End State! Theta: 337.5, Hx: 6.25, Hy:-4.5, Nosc: 174
End State! Theta: 337.5, Hx: 6.25, Hy:0.5, Nosc: 177
No End State! Theta: 337.5, Hx: 6.25, Hy:1.5, Nosc: 194
No End State! Theta: 337.5, Hx: 6.25, Hy:2.5, Nosc: 194
LastIndex:  401
End State! Theta: 337.5, Hx: 6.25, Hy:3.5, Nosc: 222
End State! Theta: 337.5, Hx: 6.25, Hy:4.5, Nosc: 51


In [60]:
colorV = (117.0/255.0,112.0/255.0,179.0/255.0)
colorO = (231.0/255.0,41.0/255.0,138.0/255.0)
colorSF = (27.0/255.0,158.0/255.0,119.0/255.0)
colorHB = (217.0/255.0,95.0/255.0,2.0/255.0)
def PlotCombined_Config_Dynamics(data,ax):
    #Data Values
    Hx      = data['Hx']
    Hy      = data['Hy']
    parHx   = data['parHx']
    parHy   = data['parHy']
    
    data.loc['vmag'] = np.hypot(data.loc['vx'],data.loc['vy'])
    minVmag = np.amin(data['vmag'])
    print('Theta = {0}: minV = {1}'.format(data.loc[0,'parThetaBW'],minVmag))
    data['nvx'] = data['vx']/data['vmag']
    data['nvy'] = data['vy']/data['vmag']
    vx      = data['vx']
    vy      = data['vy']
    nvx      = data['nvx']
    nvy      = data['nvy']

    
    #image = Construct_Image_Data(data)
    
    #Create
    
    #GENERATE FIGURE
    #Phase Space  2D (Hy vs Hx)
    ax.scatter(Hx,Hy,s=9,c='r')
    ax.quiver(parHx,parHy,nvx,nvy,scale=2,width=0.005*10,units='xy',color='k')
    ax.scatter(data['parHx'],data['parHy'],c='k',s=9,marker='s')
    
    #Add Swimmer 1 location
    Circle1 = plt.Circle((0.0,0.5),1.0,color='k', clip_on=True)
    ax.add_artist(Circle1)
    Circle2 = plt.Circle((0.0,-2.0),0.5,color='k', clip_on=True)
    ax.add_artist(Circle2)
    
    return ax
'''
endData['color'] = 0.0
for idx in range(len(endData['parThetaBW'])):
    if(endData.loc[idx,'State'] == 0):
        #Diverge
        endData.loc[idx,'color'] = (0.5,0.5,0.5)
    elif(endData.loc[idx,'State'] == 1):
        #V-Shape
        endData.loc[idx,'color'] = colorV
    elif(endData.loc[idx,'State'] == 2):
        #In-Line
        endData.loc[idx,'color'] = colorSF
    elif(endData.loc[idx,'State'] == 3):
        #Headbutt
        endData.loc[idx,'color'] = colorHB
    elif(endData.loc[idx,'State'] == 4):
        #L-Shape, but In-Line
        endData.loc[idx,'color'] = colorSF
'''
        

#Dynamics code
for Theta in ThetaList:
    print('Theta = ',Theta)
    thetaData = endData[endData['parThetaBW'] <= float(Theta)+0.01].copy()
    thetaData = thetaData[thetaData['parThetaBW'] >= float(Theta)-0.01].copy()
    thetaData = thetaData.sort_values(by=['parHx','parHy'])
    thetaData = thetaData.reset_index(drop=True)
    print(thetaData.head(10))
    #Generate a plot with net vectors at each initial location.
    #Vector color will be black atm
    fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
    ax.set_title(r'$\theta$ = {0}: $H_y$ vs. $H_x$'.format(Theta),fontsize=15,**csfont)
    ax.set_xlabel(r'$\hat{H}_x$',fontsize=12,**csfont)
    ax.set_ylabel(r'$\hat{H}_y$',fontsize=12,**csfont)
    ax.axis([-1.0,7.5,-7.5,5.0])
    ax.set_aspect('equal')
    ax = PlotCombined_Config_Dynamics(thetaData,ax)
    #2D PLOTS
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    strDir = cwd_PYTHON+"/../Figures/PhaseSpace/DynamicalSystem/".format(Theta)
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    fig.savefig(strDir+'PairDynamics_T'+Theta+'_.png')
    fig.clf()
        
    print('Theta = '+Theta+' is complete!')
    
plt.close()

Theta =  0.0
   parThetaBW  parHx  parHy   ThetaBW        Hx        Hy  Nosc  State  \
0         0.0   0.25   -6.5  0.004496  0.017257 -4.608284  17.0    2.0   
1         0.0   0.25   -5.5  6.281778  0.013585 -4.597352  11.0    2.0   
2         0.0   0.25   -4.5  6.255765 -0.034028 -4.569117   5.0    2.0   
3         0.0   0.25    4.5  6.254943  0.092991  4.570465   5.0    2.0   
4         0.0   0.75   -6.5  6.271696 -0.015420 -4.564981  18.0    2.0   
5         0.0   0.75   -5.5  6.276829 -0.008946 -4.571813  12.0    2.0   
6         0.0   0.75   -4.5  6.204668 -0.106949 -4.580080   5.0    2.0   
7         0.0   0.75    4.5  6.203651  0.249105  4.584924   5.0    2.0   
8         0.0   1.25   -6.5  0.008183 -0.001286 -4.567782  19.0    2.0   
9         0.0   1.25   -5.5  0.006361 -0.004604 -4.569575  13.0    2.0   

         vx        vy  
0 -0.013691  0.111277  
1 -0.021492  0.082059  
2 -0.056806 -0.013823  
3 -0.031402  0.014093  
4 -0.042523  0.107501  
5 -0.063245  0.077349  
6 -0

Theta = 135.0 is complete!
Theta =  157.5
   parThetaBW  parHx  parHy   ThetaBW        Hx         Hy   Nosc  State  \
0       157.5   0.25   -6.5  0.922852 -4.330396  -2.158416   54.0    1.0   
1       157.5   0.25   -5.5  0.922371 -4.329000  -2.152989   52.0    1.0   
2       157.5   0.25    3.5  3.136415  0.012158   3.455078   22.0    3.0   
3       157.5   0.25    4.5  3.144129 -0.054553  10.475741  227.0    0.0   
4       157.5   0.75   -6.5  0.922099 -4.338415  -2.153991   55.0    1.0   
5       157.5   0.75   -5.5  0.922277 -4.331856  -2.157152   52.0    1.0   
6       157.5   0.75    3.5  3.141988 -0.006686   3.456830   24.0    3.0   
7       157.5   0.75    4.5  3.140154 -0.111671  10.414626  191.0    0.0   
8       157.5   1.25   -6.5  0.921725 -4.340012  -2.155029   58.0    1.0   
9       157.5   1.25   -5.5  0.922357 -4.338154  -2.157966   56.0    1.0   

         vx        vy  
0 -0.084822  0.080400  
1 -0.088058  0.064366  
2 -0.010811 -0.002042  
3 -0.001342  0.026325  
4

Theta = 292.5 is complete!
Theta =  315.0
   parThetaBW  parHx  parHy   ThetaBW        Hx        Hy  Nosc  State  \
0       315.0   0.25   -6.5  6.263460 -0.023188 -4.602021  29.0    2.0   
1       315.0   0.25   -5.5  6.274397 -0.004563 -4.600317  23.0    2.0   
2       315.0   0.25   -4.5  0.001417  0.012466 -4.588004  18.0    2.0   
3       315.0   0.25    4.5  0.053888 -0.164061  4.604973  22.0    2.0   
4       315.0   0.75   -6.5  6.277594 -0.002833 -4.596089  30.0    2.0   
5       315.0   0.75   -5.5  0.002011  0.006344 -4.594729  24.0    2.0   
6       315.0   0.75   -4.5  0.008212  0.013892 -4.600048  19.0    2.0   
7       315.0   0.75    4.5  0.064543 -0.211295  4.544999  20.0    2.0   
8       315.0   1.25   -6.5  0.004906  0.005319 -4.568002  31.0    2.0   
9       315.0   1.25   -5.5  0.024889  0.031831 -4.567908  26.0    2.0   

         vx        vy  
0 -0.009420  0.065448  
1 -0.011068  0.039117  
2 -0.013196 -0.004889  
3 -0.018821  0.004771  
4 -0.025094  0.063464  

In [95]:
#Plot 3D scatter using Plotly for each one
#Headbutt = 3

colorV = (117.0/255.0,112.0/255.0,179.0/255.0)
colorO = (231.0/255.0,41.0/255.0,138.0/255.0)
colorSF = (27.0/255.0,158.0/255.0,119.0/255.0)
colorHB = (217.0/255.0,95.0/255.0,2.0/255.0)

endData['w'] = 0.0
for idx in range(len(endData['ThetaBW'])):
    w1 = (180.0*endData.loc[idx,'ThetaBW']/np.pi - endData.loc[idx,'parThetaBW'])/endData.loc[idx,'Nosc']
    w2 = (180.0*endData.loc[idx,'ThetaBW']/np.pi - endData.loc[idx,'parThetaBW']+360.0)/endData.loc[idx,'Nosc']
    w3 = (180.0*endData.loc[idx,'ThetaBW']/np.pi - endData.loc[idx,'parThetaBW']-360.0)/endData.loc[idx,'Nosc']
    wmin = min(abs(w1),abs(w2),abs(w3))
    endData.loc[idx,'w'] = wmin

#endData['w'] = (180.0*endData['ThetaBW']/np.pi - endData['parThetaBW'])/endData['Nosc']

df = endData.copy()

df['vmag'] = np.hypot(df['vx'],df['vy'])
df['nvx'], df['nvy'] = df['vx']/df['vmag'], df['vy']/df['vmag']
df['blank'] = 0.0


for idx in range(len(df['parThetaBW'])):
    if(df.loc[idx,'State'] == 0):
        #Diverge
        df.loc[idx,'color'] = 'rgb(127,127,127)'
    elif(df.loc[idx,'State'] == 1):
        #V-Shape
        df.loc[idx,'color'] = 'rgb(117,112,179)'
    elif(df.loc[idx,'State'] == 2):
        #In-Line
        df.loc[idx,'color'] = 'rgb(27,158,119)'
    elif(df.loc[idx,'State'] == 3):
        #Headbutt
        df.loc[idx,'color'] = 'rgb(217,95,2)'
    elif(df.loc[idx,'State'] == 4):
        #L-Shape, but In-Line
        df.loc[idx,'color'] = 'rgb(27,158,119)'

traces = [0 for a in range(3)]

traces[0] = go.Cone(
    x=(df['parThetaBW'])/360.0,
    y=(df['parHy']+6.5)/(6.5+4.5),
    z=(df['parHx']+0.25)/(0.25+6.25),
    u=df['w'],
    v=df['nvy'],
    w=df['nvx'],
    #color=df['color'],
    #colorscale='Viridis',
    #sizemode="absolute",
    sizemode="scaled",
    #anchor='tail',
    #sizeref=100.0,
    sizeref=4)

traces[1] = go.Scatter3d(
    x=df['ThetaBW']/(2.0*np.pi),
    y=(df['Hy']+6.5)/(6.5+4.5),
    z=(df['Hx']+0.25)/(0.25+6.25),
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=6,
        #color=endData['color'],                # set color to an array/list of desired values
        color=df['color'],
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=10)
    )
)

traces[2] = go.Scatter3d(
    x=[0.0,0.0],
    y=[(0.5+6.5)/(6.5+4.5),(-2.0+6.5)/(6.5+4.5)],
    z=[(0.25)/(0.25+6.25),(0.25)/(0.25+6.25)],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=18,
        #color=endData['color'],                # set color to an array/list of desired values
        color='Black',
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=10)
    )
)

fig = go.Figure(data = traces, layout=layout)

layout = go.Layout(
    #title='Re10: 3D Phase Diagram',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    showlegend=False,
)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene = dict(
                    xaxis_title=r'theta',
                    yaxis_title=r'Hy (R)',
                    zaxis_title=r'Hx (R)'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=8, range=[0,1],),
        yaxis = dict(nticks=8, range=[0,1],),
        zaxis = dict(nticks=8, range=[0,1],),
        aspectmode='cube'))


fig.show()